<a href="https://colab.research.google.com/github/srpantano/deep-learningaz/blob/master/autos_regressao_um_valor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lê o dataset a partir do Kaggle
Importate importar antes da chave de API (arquivo json)

In [0]:
from google.colab import files

!pip install -q kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d orgesleka/used-cars-database

 94% 17.0M/18.0M [00:00<00:00, 21.4MB/s]
100% 18.0M/18.0M [00:00<00:00, 40.7MB/s]


In [8]:
!pwd

/content


In [29]:
import zipfile

file = zipfile.ZipFile('used-cars-database.zip', 'r') 
file.extract('autos.csv')

'/content/autos.csv'

In [0]:
import pandas as pd
train_dataset = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [0]:
#display(train_dataset)

### Remove colunas da tabela

In [0]:
train_dataset = train_dataset.drop('dateCrawled', axis = 1) # axis=1 apaga a coluna inteira
train_dataset = train_dataset.drop('dateCreated', axis = 1)
train_dataset = train_dataset.drop('nrOfPictures', axis = 1)
train_dataset = train_dataset.drop('postalCode', axis = 1)
train_dataset = train_dataset.drop('lastSeen', axis = 1)

### Conta as ocorrências

In [0]:
train_dataset['name'].value_counts()

In [0]:
train_dataset = train_dataset.drop('name', axis = 1) # Muitos dados distintos (variabilidade), sem significado
train_dataset = train_dataset.drop('seller', axis = 1)
train_dataset = train_dataset.drop('offerType', axis = 1)

### Tratamento dos dados

In [0]:
i1 = train_dataset.loc[train_dataset.price <= 10] #Verifica quais automóveis tem o preço abaixo de 10
#print(train_dataset.price.mean()) #Exibe a média dos preços
train_dataset = train_dataset[train_dataset.price > 10] # Somente os que tem preço acima de 10
i1 = train_dataset.loc[train_dataset.price > 350000] #Verifica quais automóveis tem o preço acima de 350k
train_dataset = train_dataset[train_dataset.price < 350000]

In [0]:
#train_dataset.loc[pd.isnull(train_dataset['vehicleType'])]

### Preenche os nulos



1.   Conta as quantidades;
2.  Preenche os nulos com os dados mais recorrentes.



In [0]:
#train_dataset['vehicleType'].value_counts() #Conta os tipos de veículos
#train_dataset['gearbox'].value_counts()
#train_dataset['model'].value_counts()
#train_dataset['fuelType'].value_counts()
#train_dataset['notRepairedDamage'].value_counts()

In [0]:
# Substitui os nulos com dados mais recorrentes (new_values)
new_values = {'vehicleType' : 'limousine', 'gearbox' : 'manuell', 'model' : 'golf', 
          'fuelType' : 'benzin', 'notRepairedDamage' : 'nein'}
train_dataset = train_dataset.fillna(new_values)
x_train = train_dataset.iloc[:, 1:13].values #Todas as linhas, nas colunas 1 a 13
price_real = train_dataset.iloc[:, 0].values #Todas as linhas, coluna 0

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [0]:
labelencoder = LabelEncoder()
x_train[:, 0] = labelencoder.fit_transform(x_train[:, 0])
x_train[:, 1] = labelencoder.fit_transform(x_train[:, 1])
x_train[:, 3] = labelencoder.fit_transform(x_train[:, 3])
x_train[:, 5] = labelencoder.fit_transform(x_train[:, 5])
x_train[:, 8] = labelencoder.fit_transform(x_train[:, 8])
x_train[:, 9] = labelencoder.fit_transform(x_train[:, 9])
x_train[:, 10] = labelencoder.fit_transform(x_train[:, 10])

In [0]:
x_train[0:20] #vinte primeiros dados

### Transforma o que é 0, 1, ... 3 em 0 0 1, 0 ...
0 em 1 0 0 0 </br>
1 em 0 1 0 0</br>
2 em 0 0 1 0</br>
3 em 0 0  0 1 </br>

In [40]:
onehotencoder = OneHotEncoder(categorical_features = [0,1,3,5,8,9,10])
x_train = onehotencoder.fit_transform(x_train).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
x_train[0:20, 0:9]

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [0]:
model = Sequential()
model.add(Dense(units = 158, activation = 'relu', input_dim = x_train.shape[1]))
model.add(Dense(units = 158, activation = 'relu'))
model.add(Dense(units = 1, activation = 'linear')) #linear não faz nenhum cálculo
model.compile(loss = 'mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [48]:
x_train.shape

(359291, 316)

In [45]:
price_real.shape

(359291,)

In [0]:
# O mean_absolute_error indica que ele pode errar em X vezes em euros
model.fit(x_train, price_real, batch_size=300, epochs=100) 

In [53]:
yhat = model.predict(x_train)
print(price_real.mean())
print(yhat.mean())

5916.833945186492
5714.881
